In [46]:
%pylab 
from pygadgetreader import *
from octopus import density
from numpy import linalg
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [111]:
# Measuring the Warp of the MW

In [3]:
# From http://stackoverflow.com/questions/22867620/putting-arrowheads-on-vectors-in-matplotlibs-3d-plot
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [141]:
disk_pos = readsnap('../data/LMCMW/MWmLMC6/H1/MWfLMC6H1_119.hdf5', 'pos', 'disk')
disk_vel = readsnap('../data/LMCMW/MWmLMC6/H1/MWfLMC6H1_119.hdf5', 'vel', 'disk')
disk_mass = readsnap('../data/LMCMW/MWmLMC6/H1/MWfLMC6H1_119.hdf5', 'mass', 'disk')
disk_pot = readsnap('../data/LMCMW/MWmLMC6/H1/MWfLMC6H1_119.hdf5', 'pot', 'disk')

Returning DISK  Positions
Returning DISK  Velocities
Returning DISK  Mass
Returning DISK  Potential


In [142]:
min_pot = np.sort(disk_pot)

In [143]:
x_cm = np.zeros(100)
y_cm = np.zeros(100)
z_cm = np.zeros(100)

vx_cm = np.zeros(100)
vy_cm = np.zeros(100)
vz_cm = np.zeros(100)


for i in range(100):
    index = np.where(disk_pot == min_pot[i])[0]
    x_cm[i] = disk_pos[index,0]
    y_cm[i] = disk_pos[index,1]
    z_cm[i] = disk_pos[index,2]
    vx_cm[i] = disk_vel[index,0]
    vy_cm[i] = disk_vel[index,1]
    vz_cm[i] = disk_vel[index,2]

In [144]:
def cm(x, y, z, vx, vy, vz):
    xcm = np.sum(x) / len(x)
    ycm = np.sum(y) / len(y)
    zcm = np.sum(z) / len(z)
    vxcm = np.sum(vx) / len(vx)
    vycm = np.sum(vy) / len(vy)
    vzcm = np.sum(vz) / len(vz)
    return xcm, ycm, zcm, vxcm, vycm, vzcm

In [145]:
xcm, ycm, zcm, vxcm, vycm, vzcm = cm(x_cm, y_cm, z_cm, vx_cm, vy_cm, vz_cm)

In [146]:
scatter(disk_pos[:,1], disk_pos[:, 2], s=0.1)
scatter(y_cm, z_cm)


In [147]:
disk_cut = np.where((np.abs(disk_pos[:,0]-xcm)<40) & (np.abs(disk_pos[:,1]-ycm)<40) & (np.abs(disk_pos[:,2]-zcm)<10))[0]

In [148]:
X_d = disk_pos[disk_cut,0] - xcm
Y_d = disk_pos[disk_cut,1] - ycm
Z_d = disk_pos[disk_cut,2] - zcm

Vx_d = disk_vel[disk_cut,0] - vxcm
Vy_d = disk_vel[disk_cut,1] - vycm
Vz_d = disk_vel[disk_cut,2] - vzcm

XYZ_d = np.zeros((len(X_d), 3))
XYZ_d[:,0] = X_d
XYZ_d[:,1] = Y_d
XYZ_d[:,2] = Z_d

vXYZ_d = np.zeros((len(X_d), 3))
vXYZ_d[:,0] = Vx_d
vXYZ_d[:,1] = Vy_d
vXYZ_d[:,2] = Vz_d

In [149]:
subplot(1, 2, 1)
scatter(Y_d, Z_d, s=0.1)
axis('equal')

subplot(1, 2, 2)
scatter(XYZ_d[:,0], XYZ_d[:,1], s=0.1)

#xlim(-30, 30)
#ylim(-20, 0)

In [58]:
#disk_dens = density.grid(X_d, Y_d, Z_d, 50)

In [59]:
#imshow(np.log10(disk_dens.T), origin='lower', cmap='viridis')

In [49]:
disk_densYZ = density.grid(Y_d, Z_d, X_d, 100)

In [61]:
#imshow(np.log10(disk_densYZ.T), origin='lower', cmap='viridis')
#xlim(50, 80)
#ylim(20, 50)

In [133]:
def angular_m(xyz, vxyz):
    r_c_p = ([np.cross(xyz[i], vxyz[i]) for i in range(len(xyz))])
    J_x_i = np.zeros(len(r_c_p))
    J_y_i = np.zeros(len(r_c_p))
    J_z_i = np.zeros(len(r_c_p))
    for i in range(len(r_c_p)):
        J_x_i[i] = r_c_p[i][0]
        J_y_i[i] = r_c_p[i][1]
        J_z_i[i] = r_c_p[i][2]
    J_x = np.sum(J_x_i)
    J_y = np.sum(J_y_i)
    J_z = np.sum(J_z_i)
    J = np.array([J_x, J_y, J_z])
    return J/ linalg.norm(J) 

In [134]:
R = np.sqrt(XYZ_d[:,0]**2.0 + XYZ_d[:,1]**2.0)
part_select = np.where((np.abs(R)<40) & (np.abs(XYZ_d[:,2])<10))[0]
J_t = angular_m(XYZ_d[part_select], vXYZ_d[part_select])
phi = np.arccos(J_t[0] / linalg.norm(J_t))
print phi
    

1.56712983206


In [135]:
rad2deg(phi)

89.789925325798521

In [110]:
phi*180 / (np.pi)

89.830361662408748

In [21]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
a = Arrow3D([0,J[0]],[0,J[1]],[0,J[2]], mutation_scale=20, lw=1, arrowstyle="-|>", color="k", linestyle="dashed")
ax.scatter3D(XYZ_d[:,0], XYZ_d[:,1], XYZ_d[:,2], s=0.1)
ax.add_artist(a)
ax.set_zlim(-20, 20)
ax.set_ylim(-30, 30)
ax.set_xlim(-30, 30)

(-30, 30)

In [22]:
scatter(XYZ_d[:,1], XYZ_d[:,2], s=0.1)

# h measurement

## Using eq. 7 of Levine 2006

In [150]:
def mean_z(x, y, z):
    Nx = 2*(max(x) - min(x))
    Ny = 2*(max(y) - min(y))
    print Nx, Ny
    x_grid = np.linspace(min(x), max(x), int(Nx))
    y_grid = np.linspace(min(y), max(y), int(Ny))
    z_mean = np.zeros((Nx, Ny))
    for i in range(1,len(x_grid)):
        for j in range(1,len(y_grid)):
            z_cut = np.where(((x-x_grid[i])**2.0 +(y-y_grid[j])**2.0)<1.0)[0]
            z_mean[i][j] = np.mean(z[z_cut])
    return z_mean
            

In [151]:
Z_test = mean_z(XYZ_d[:,0], XYZ_d[:,1], XYZ_d[:,2])

153.107559204 155.437446594


In [154]:
imshow((Z_test), cmap='viridis')
colorbar()

In [181]:
def rho_disk(x, y, z, delta):
    x_grid = np.linspace(min(x), max(x), delta)
    y_grid = np.linspace(min(y), max(y), delta)
    z_grid = np.linspace(min(z), max(z), delta)
    rho_weight = np.zeros((delta, delta))
    rho = np.zeros(delta)
    for i in range(1,len(x_grid)):
        for j in range(1,len(y_grid)):
            for k in range(1,len(z_grid)):
                index = np.where((x<x_grid[i]) & (x>x_grid[i-1]) & (y<y_grid[j]) & (y>y_grid[j-1]) & (z<z_grid[k]) & (z>z_grid[k-1]))[0]
                N = len(index)
                rho[k] = N / delta**2.0 * z[k] 
            rho_weight[i][j] = sum(rho)
    return rho_weight

In [170]:
rho_t = rho_disk(XYZ_d[:,0], XYZ_d[:,1], XYZ_d[:,2], 50)

In [171]:
imshow(rho_t[:,18,:])

In [178]:
def sigma(x, y, z, delta):
    x_grid = np.linspace(min(x), max(x), delta)
    y_grid = np.linspace(min(y), max(y), delta)
    S = np.zeros((delta, delta))
    for i in range(1,len(x_grid)):
        for j in range(1,len(y_grid)):
            index = np.where((x<x_grid[i]) & (x>x_grid[i-1]) & (y<y_grid[j]) & (y>y_grid[j-1]))[0]
            N = len(index)
            S[i][j] = N / delta**2.0 
    return S
    
    

In [192]:
sigma_t = sigma(XYZ_d[:,0], XYZ_d[:,1], XYZ_d[:,2], 500)

In [193]:
imshow(np.log10(sigma_t), extent=[min(XYZ_d[:,0]), max(XYZ_d[:,0]), min(XYZ_d[:,1]), max(XYZ_d[:,1])], cmap='viridis')